## tf-nlr: Neural Lumigraph Rendering in TensorFlow 2 ##

<div style='text-align:center'><img src='img/M2.gif' alt='Rendered gif of NLR scene M2.' /></div>

This notebook shows how to use `tf-nlr`, an implementation of the neural pipeline of "Neural Lumigraph Rendering" as described in [Kellnhofer et al. (2021)](https://arxiv.org/abs/2103.11571).

To run this notebook, please download the free [NLR dataset](https://drive.google.com/file/d/1BBpIfrqwZNYmG1TiFljlCnwsmL2OUxNT/view) and extract it in a new folder called `data` in the root directory of `tf-nlr`.

In [ ]:
import os
import time
import datetime
from argparse import Namespace

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from model import loss
from model.nlr import NeuralLumigraph

from lib.sphere_tracer import SphereTracer
from lib.data import Data
from lib.math import dot, sphere_data, gen_3d_noise, compute_gradients, normalize_vectors

from train import train

### Load data ###

 - The `lib.data.Data` object packages data loading and ray generation.
 - Initialize a `Data` object with the command `Data('path/to/dataset', img_ratio)`.
 - Note that it must be bound to the `model.nlr.NeuralLumigraph` object with the `bind_data` method, as we show below.

In [ ]:
nlr_data = Data('./data/nlr_dataset/L1', img_ratio=5)

In [ ]:
nlr_data.compute_rays(scene_radius_scale=.7)

In [ ]:
nlr_dataset = nlr_data.compute_dataset(v_img=-1)

### Create a neural lumigraph model ###

<div style='text-align:center'><img src='img/L1.gif' alt='Rendered gif of NLR scene L1.' style='width:120px;height:auto' /></div>

 - With `model.nlr.NeuralLumigraph.pretrain`, you can initialize the neural SDF to a sphere of radius 0.5. If you don't want to, skip to [this cell](#train-from-scratch).
 - This requires some learning rate schedule or otherwise manual adjustment of the learning rate.

In [ ]:
nlr = NeuralLumigraph(omega=(30, 30), hidden_omega=(30, 30))
nlr.compile(s_lr=tf.keras.optimizers.schedules.ExponentialDecay(1e-4, 1000, .5, staircase=True), e_lr=1e-4)
nlr.pretrain(radius=.5)

You can also load a model already trained on NLR data. You'll want to pass `permute_inputs=True` for this.

In [ ]:
nlr = NeuralLumigraph(omega=(30,30), hidden_omega=(30,30), permute_inputs=True)
nlr.load_model('h5/L1')

<span id='train-from-scratch'>If you want to train from scratch, start here.</span>

In [ ]:
nlr = NeuralLumigraph(s_h5='h5/pretrain/S.h5', omega=(30,30), hidden_omega=(30,30))
nlr.compile(s_lr=tf.keras.optimizers.schedules.ExponentialDecay(1e-6, 30000, .5, staircase=True), 
            e_lr=tf.keras.optimizers.schedules.ExponentialDecay(1e-4, 30000, .5, staircase=True))

### Instantiate the sphere tracer ###

 - NLR uses a bidirectional sphere tracer that converges points which reach within a small threshold of zero.
 - Before the official code for [MetaNLR++](https://github.com/alexanderbergman7/metanlrpp) was released, back in July 2021, `tf-nlr` used a sphere tracer which more closely resembled that of [IDR](https://github.com/lioryariv/idr/blob/main/code/model/ray_tracing.py), which MetaNLR++ also appears to use. However, since the MetaNLR++ code contains the version of the sphere tracer used in the original NLR, we've since transferred over to a translation of their code. Our original sphere tracer can be found as the `alt_trace` method of the `lib.sphere_tracer.SphereTracer` object.
 - The `SphereTracer` object should be bound to the model with the `bind_tracer` method.

In [ ]:
sphere_tracer = SphereTracer(sphere_trace_n=16)
nlr.bind_tracer(sphere_tracer)

### Bind data and render ###

Bind the data object as below.

In [ ]:
nlr.bind_data(nlr_data)

 - Here are three examples of how to use a `NeuralLumigraph` object to render views.
 - The NLR object supports rendering RGB, depth, and normal maps.
 - By default, the `render` and `write_img` methods render the view indexed by the `v_img` attribute of the bound `Data` object. This `v_img` attribute is set to `-1`, i.e. the last image in the dataset, unless passed as a keyword argument into the `compute_dataset` method or otherwise set as an attribute to the `NeuralLumigraph` object. Alternatively, you can pass a `v_img` keyword argument to these methods to specify you want to render a different image.

With `write_img`, you don't have to do any of the plotting work; just call the method. It returns a `matplotlib.pyplot` object, so you can call `show` on it.

In [ ]:
rendered_view, _ = nlr.write_img(v_img=16, compute_depth_img=True, compute_normal_img=True, write_to_file=False)
rendered_view.show()

 - Alternatively, you can use `render` to have more control over how the result is plotted.
 - Pass functions as `transform_rays_o` and `transform_rays_d` keyword arguments to synthesize novel views.

In [ ]:
for img_number in range(-1, -len(nlr_data.img_tensors)-1, -1):
    final_img = nlr.render(v_img=img_number)

    f = plt.figure(figsize=(10,4))
    plt.subplot(121)
    plt.imshow(tf.reshape(final_img, nlr.data.img_tensors[img_number][0].shape))
    plt.subplot(122)
    plt.imshow(nlr_data.img_tensors[img_number][0])
    plt.show()

In [ ]:
for i in range(4):
    tf.print(f'Panning left... please wait! {i}/3')
    final_img = nlr.render(v_img=16, transform_rays_o=lambda x : x + tf.constant([-.05*i, 0., 0.]), verbose=False)
    plt.imshow(tf.reshape(final_img, nlr.data.img_tensors[16][0].shape))
    plt.show()

### Standard training loop ###

NLR uses an initial learning rate of $1 \times 10^{-4}$.

In [ ]:
# use these learning rates!
nlr.sdf.optimizer.learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(1e-6, 30000, .5, staircase=True)
nlr.e.optimizer.learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(1e-4, 30000, .5, staircase=True)

Below are training and validation parameters; the default values here should work in most cases, but you may have to adjust the device options depending on your hardware constraints.

In [ ]:
opt = dict(
    training = dict(
        # general
        epochs = 750, # train for this many epochs
        batch_size = 50000, # split the data into batches of this size
        shuffle = 0, # shuffle the data with this buffer size (0 means full dataset size)

        nan_exception = True, # raise exception on loss NaN?
        print_losses = True, # print losses every step?
        print_times = True, # print times?
        print_clear = True, # clear output every epoch?

        # checkpoints
        checkpoints = dict(
            write_checkpoints = True, # save checkpoints?
            checkpoint_steps = 0, # checkpoint every this many steps; if 0, checkpoint on validation
            checkpoint_path = 'h5',
        ),
        
        # TensorBoard
        tensorboard = dict(
            write_summaries=True, # write to TensorBoard?
            losses=['l_r', 'l_m', 'l_e'], # which losses to write
            log_path='logs', # TensorBoard log path
        ),
        
        # mask loss hyperparameters
        mask_loss = dict(
            alpha_increase = 250, # increase alpha every this many epochs
            alpha_ratio = 2., # multiply alpha by this value
            alpha = 50., # initial alpha value
            num_samples = 80, # number of samples along ray to find minimal SDF value
            batch_sampling = True, # if true, does two batches for sampling
        ),

        # loss weights
        loss_weights = dict(
            w_e = 1e-1, # eikonal weight
            w_m = 1e2, # mask loss weight
            w_s = 1e-2, # angular linearization weight
        ),

        # device options
        device = dict(
            diff_sphere_tracing_device = '/gpu:0', # device for the differentiable sphere tracing step
            get_features_device = '/gpu:0', # device for recomputing normals and getting feature vectors
            appearance_forward_device = '/gpu:0', # device for forward-passing to E
            sampling_device = '/gpu:0', # device for ray sampling
            l_r_device = '/gpu:0', # device for color loss
            l_s_device = '/gpu:0', # device for angular smoothness loss
            l_e_device = '/cpu:0', # device for eikonal loss
            l_m_device = '/gpu:0', # device for soft mask loss
            optim_device = '/gpu:0', # device for optimizer step
        ),

        # validation parameters
        validation = dict(
            validate = True, # render validation image?
            validation_step = 0, # which step to validate after?
            validation_epochs = 1, # validate every this many epochs
            compute_depth_img = True, # render depth map?
            compute_normal_img = True, # render image with normal map?
            verbose = True, # verbose validation?
            validation_out_path = 'out.png', # save validation image to this path
            ipy_show = True, # if True, calls matplotlib.pyplot.show instead of saving to file
        ),
    ),
    
    rendering = dict(
        light_dir = [.3202674, -0.91123605, -0.25899315], # lighting direction for normal image; if None, return RGB normal map
        normal_bias = 70, # brightness parameter for normal image
    ),
)

opt = Namespace(**opt)

Here's the main training loop.

In [ ]:
train(opt, nlr=nlr, notebook=True)